In [1]:
import math
import numpy as np
import pandas as pd

from itertools import combinations_with_replacement
from tqdm import tqdm
from constants import *

# Methods + Hardcoded

In [2]:
## Functions

def cleanup_rares(l):
    cleaned_l = []
    
    for sub_l in l:
        add = True
        uniques, counts = np.unique(sub_l, return_counts=True)
        
        for u, c in zip(uniques, counts):
            if u in rares and c > 1:
                add=False
                
        if add:
            cleaned_l.append(sub_l)
        
    return cleaned_l

def condense_types(l):
    cleaned_l = []
    
    for sub_l in l:
        new_l = [replace[a] if a in replace else a for a in sub_l]
        if new_l not in cleaned_l:
            cleaned_l.append(new_l)
            
    return cleaned_l

def get_leavings(comb):
    leavings = []
    for a in comb:
        if '/' in a:
            a = a.split('/')[0]
        leavings.append(animal_dict[a])
    
    guaranteed, bonus = zip(*leavings)
    return guaranteed, bonus

def rarity_value(a_list):
    rarity_dict = {
        'C':0,
        'T':1,
        'W':2,
        'TW':3
    }
    
    total = 0
    
    for a in a_list:
        spl = a.split('/')
        if len(spl) > 1:
            rarity1 = list(df1.loc[df1['Animal'] == spl[0]]['Type'])[0]
            rarity2 = list(df1.loc[df1['Animal'] == spl[1]]['Type'])[0]
            
            rarity1 = rarity_dict[rarity1]
            rarity2 = rarity_dict[rarity2]
            total += min(rarity1, rarity2)
        else:
            rarity1 = list(df1.loc[df1['Animal'] == spl[0]]['Type'])[0]
            rarity1 = rarity_dict[rarity1]
            total += rarity1
    return total/len(a_list)

def print_results(a_list, totals, rarity_val):
    
    g, b = dict(), dict()
    
    print('--Pasture | Rarity: {}--'.format(rarity_val))
    for a in sorted(a_list):
        a0 = a.split('/')[0]
        
        l1 = animal_dict[a0][0]
        l2 = animal_dict[a0][1]
        
        print(f'| {a} - ({l1}/{l2})')
        
        if l1 not in g.keys():
            g[l1] = 1
        else:
            g[l1] += 1
        
        if l2 not in b.keys():
            b[l2] = 1
        else:
            b[l2] += 1
        
    print('--Avg Leavings per Day')
    for t in totals.keys():
        print(f'| {t}-{round(totals[t],1)} | Guaranteed: {g[t]} | Bonus: {b[t]}')
        
def sort_dict(d):
    return {k: v for k, v in reversed(sorted(d.items(), key=lambda item: item[1]))}

# Setup

In [3]:
# Animals
df1 = pd.read_csv('animals.csv')
# df1 = df1.loc[~df1['Type'].isin(['C'])]
animal_dict = {}
for _, r in df1.iterrows():
    animal_dict[r['Animal']] = [r['Leaving 1'], r['Leaving 2']]

# Recipes + fixes
df2 = pd.read_csv('recipes2.csv')
df2['Leavings'] = [list(x[1:-1].split(', ')) for x in df2['Leavings']]
df2['Number'] = [[int(v) for v in list(x[1:-1].split(', '))] for x in df2['Number']]

df1.head(len(df1))

,Animal,Leaving 1,Leaving 2,Type
0,Alligator,Claw,Fang,TW
1,Apkallu,Fleece,Egg,C
2,Apkallu of Paradise,Egg,Fleece,T
3,Aurochs,Milk,Horn,C
4,Beachcomb,Carapace,Claw,TW
5,Black Chocobo,Feather,Fur,W
6,Blue Back,Egg,Feather,C
7,Chocobo,Fur,Feather,C
8,Coblyn,Fang,Carapace,C
9,Dodo,Egg,Feather,C


In [4]:
df2.head(len(df2))

,Name,Time,Leavings,Number,Freq,Notes
0,Culinary Knife,4,[Claw],[1],1.00,NaN
1,Brush,4,[Fur],[1],1.00,NaN
2,Boiled Egg,4,[Egg],[1],0.50,NaN
3,Earrings,4,[Fang],[1],1.00,NaN
4,Butter,4,[Milk],[1],1.00,NaN
5,Tunic,6,[Fleece],[2],0.50,NaN
6,Hora,6,[Carapace],[2],0.50,NaN
7,Sweet Popoto,6,[Milk],[1],0.25,NaN
8,Caramels,6,[Milk],[2],0.25,NaN
9,Cavalier's Hat,6,[Feather],[2],0.50,NaN


# Recipe Material Rater (Regardless of Usefulness)

In [5]:
# Calculate Total Materials needed for 1 of each craft
recipes = df2.copy()

max_weekly = {
    'Carapace': 58,
    'Claw': 45,
    'Egg': 23,
    'Fang': 51,
    'Feather': 20,
    'Fleece': 36,
    'Fur': 40,
    'Horn': 18,
    'Milk': 37,
}       
print(sort_dict(max_weekly))

avg_weekly = {
    'Carapace': 11.7,
    'Claw': 10.1,
    'Egg': 3.7,
    'Fang': 10.9,
    'Feather': 2.4,
    'Fleece': 8.2,
    'Fur': 8.3,
    'Horn': 3.4,
    'Milk': 7.9,
}       
print(sort_dict(avg_weekly))

# Calculate Thresholds by assuming you collect the minimum amount over the course of a week
# max_thresholds = sort_dict(max_weekly)
max_thresholds = sort_dict(avg_weekly)
# Carapace >= Fur, Fang, Claw > Milk, Fleece >> Egg, Feather, Horn
for k,v in max_thresholds.items():
    max_thresholds[k] = round(v/7,1)
#     max_thresholds[k] = math.ceil(v/7)

max_thresholds, round(sum(max_thresholds.values()),1)

{'Carapace': 58, 'Fang': 51, 'Claw': 45, 'Fur': 40, 'Milk': 37, 'Fleece': 36, 'Egg': 23, 'Feather': 20, 'Horn': 18}
{'Carapace': 11.7, 'Fang': 10.9, 'Claw': 10.1, 'Fur': 8.3, 'Fleece': 8.2, 'Milk': 7.9, 'Egg': 3.7, 'Horn': 3.4, 'Feather': 2.4}


({'Carapace': 1.7,
  'Fang': 1.6,
  'Claw': 1.4,
  'Fur': 1.2,
  'Fleece': 1.2,
  'Milk': 1.1,
  'Egg': 0.5,
  'Horn': 0.5,
  'Feather': 0.3},
 9.5)

In [35]:
c_thresholds = {}
for k in max_weekly.keys():
    c_thresholds[k] = (max_weekly[k] + avg_weekly[k]) / 2
    
print(sort_dict(c_thresholds), round(sum(c_thresholds.values()),1))
    
for k,v in c_thresholds.items():
    c_thresholds[k] = round(v/7,1)
#     max_thresholds[k] = math.ceil(v/7)
    
sort_dict(c_thresholds), round(sum(c_thresholds.values()),1)

{'Carapace': 34.85, 'Fang': 30.95, 'Claw': 27.55, 'Fur': 24.15, 'Milk': 22.45, 'Fleece': 22.1, 'Egg': 13.35, 'Feather': 11.2, 'Horn': 10.7} 197.3


({'Carapace': 5.0,
  'Fang': 4.4,
  'Claw': 3.9,
  'Fur': 3.4,
  'Milk': 3.2,
  'Fleece': 3.2,
  'Egg': 1.9,
  'Feather': 1.6,
  'Horn': 1.5},
 28.1)

## Manual C_thresholds

In [7]:
# c_thresholds = {
#     'Fur': 4.6,
#     'Carapace': 4.3,
#     'Fang': 3.7,
#     'Claw': 3.4,
#     'Milk': 3,
#     'Fleece': 3,
#     'Horn': 1.8,
#     'Egg': 1.5,
#     'Feather': 1.3,
# }
# sort_dict(c_thresholds), round(sum(c_thresholds.values()),1)

# Calculations

In [15]:
thresholds = c_thresholds
# thresholds = max_thresholds

base_yield = {
    'Carapace' : 4,
    'Fur': 3,
    'Fang': 3,
    'Claw': 3,
    'Milk': 2,
    'Fleece': 2,
    'Egg': 1,
    'Horn': 1,
    'Feather': 1
}
bonus_yield = dict()

for k,v in thresholds.items():
    # Ensure we hit some guaranteed amount
    bonus_yield[k] = max(0, round(thresholds[k] - base_yield[k],1))

base_yield, bonus_yield, sum(base_yield.values()), sum(bonus_yield.values())

({'Carapace': 4,
  'Fur': 3,
  'Fang': 3,
  'Claw': 3,
  'Milk': 2,
  'Fleece': 2,
  'Egg': 1,
  'Horn': 1,
  'Feather': 1},
 {'Carapace': 1.0,
  'Claw': 0.9,
  'Egg': 0.9,
  'Fang': 1.4,
  'Feather': 0.6,
  'Fleece': 1.2,
  'Fur': 0.4,
  'Horn': 0.5,
  'Milk': 1.2},
 20,
 8.1)

In [18]:
combs = dict()
total = 0
for k in base_yield.keys():
    temp = df1.loc[df1['Leaving 1'] == k]
#     print(k,'\n',temp,'\n')
    if base_yield[k] > 1:
        # GET LIST
        l = list(combinations_with_replacement(temp['Animal'], base_yield[k]))
        
        # CLEAN OUT EXTRA RARES
        l = cleanup_rares(l)
        l = condense_types(l)
        
        if total == 0:
            total = len(l)
        elif total > 1:
            total *= len(l)
        
        # Add cleaned out combinations to list
        combs[k] = l
    else:
        l = [[a] for a in temp['Animal']]
        l = condense_types(l)
        
        if total == 0:
            total = len(l)
        elif total > 1:
            total *= len(l)
        
        combs[k] = l
        
combs

{'Carapace': [['Beachcomb/Glyptodon Pup',
   'Beachcomb/Glyptodon Pup',
   'Beachcomb/Glyptodon Pup',
   'Beachcomb/Glyptodon Pup'],
  ['Beachcomb/Glyptodon Pup',
   'Beachcomb/Glyptodon Pup',
   'Beachcomb/Glyptodon Pup',
   'Yellow Coblyn/Morbol'],
  ['Beachcomb/Glyptodon Pup',
   'Beachcomb/Glyptodon Pup',
   'Yellow Coblyn/Morbol',
   'Yellow Coblyn/Morbol']],
 'Fur': [['Chocobo', 'Chocobo', 'Chocobo'],
  ['Chocobo', 'Chocobo', 'Island Doe'],
  ['Chocobo', 'Chocobo', 'Island Stag'],
  ['Chocobo', 'Chocobo', 'Lemur/Star Marmot'],
  ['Chocobo', 'Chocobo', 'Amethyst Spriggan'],
  ['Chocobo', 'Island Doe', 'Island Doe'],
  ['Chocobo', 'Island Doe', 'Island Stag'],
  ['Chocobo', 'Island Doe', 'Lemur/Star Marmot'],
  ['Chocobo', 'Island Doe', 'Amethyst Spriggan'],
  ['Chocobo', 'Island Stag', 'Lemur/Star Marmot'],
  ['Chocobo', 'Island Stag', 'Amethyst Spriggan'],
  ['Chocobo', 'Lemur/Star Marmot', 'Lemur/Star Marmot'],
  ['Chocobo', 'Lemur/Star Marmot', 'Amethyst Spriggan'],
  ['Island 

In [21]:
def evaluate_pasture(comb, threshold_bonus, chance=0.8):
    guaranteed, bonus = get_leavings(comb)
    
    # Ensure we're getting 2 of each material guaranteed at minimum
    #   as a way to combat bad RNG
    g_labels, g_num = np.unique(guaranteed, return_counts=True)
    b_labels, b_num = np.unique(bonus, return_counts=True)
    b_num = np.multiply(b_num, chance)
    
    # Get total
    g_total = {k:v for k, v in zip(g_labels, g_num)}
    b_total = {k:v for k, v in zip(b_labels, b_num)}
    total = dict()
    
    for i, _ in enumerate(g_labels):
        total[g_labels[i]] = g_num[i] 
    for i, _ in enumerate(b_labels):
        total[b_labels[i]] += round(b_num[i],1)
        
    total = sort_dict(total)
    
    # We already guarantee the base yield when generating the initial lists. We just need to calculate the bonus yield
    
    # If any of our totals are less than the base yield threshold for any leaving, break
    for key in total.keys():
        # If we're expecting a bonus and the list doesn't offer one, return 1
        if key not in b_total.keys() and key in threshold_bonus.keys():
            return total, 1
        # If we're expecting a bonus and the list doesn't offer enough, return 2
        if b_total[key] < threshold_bonus[key]:
            return total, 2

    # Finally, make sure we bias priorities
    # That is to say: Early Filter out certain things
    total_keys = list(total.keys())
    return total, 0
#     if all(total[k] < 2 for k in ['Horn', 'Feather', 'Egg']):
#         return total, 0
#     else:
#         return total, 3

In [22]:
best_setups = {}
rarity_vals = {}
errs = {
    0:0,
    1:0,
    2:0,
    3:0
}

with tqdm(total=total) as pbar:
    for c1 in combs['Carapace']:
        for c2 in combs['Fur']:
            for c3 in combs['Fang']:
                for c4 in combs['Fleece']:
                    for c5 in combs['Claw']:
                        for c6 in combs['Milk']:
                            for c7 in combs['Horn']:
                                for c8 in combs['Feather']:
                                    for c9 in combs['Egg']:
                                        l = c1 + c2 + c3 + c4 + c5 + c6 + c7 + c8 + c9
                                        t, meets = evaluate_pasture(l, bonus_yield, chance=0.5)

                                        errs[meets] += 1
                                        if meets == 0:
                                            r = round(rarity_value(l),2)
                                            size =  len(np.unique(l))
                                            if size not in best_setups.keys():
                                                best_setups[size] = [[l, t, r]]
                                            else:
                                                best_setups[size].append([l, t, r])
                                            if r not in rarity_vals.keys():
                                                rarity_vals[r] = [[l,t,r]]
                                            else:
                                                rarity_vals[r].append([l,t,r])

                                        pbar.update(1)
        
print(errs)
for key in sorted(best_setups.keys()):
    print(key, len(best_setups[key]))
print('')
for key in sorted(rarity_vals.keys()):
    print(key, len(rarity_vals[key]))

try:
    min_v, max_v = min(best_setups.keys()), max(best_setups.keys())
except:
    print('Given Thresholds Combined with Given food do not give good enough results')

100%|██████████████████████████████████████████████████████████████████████| 2488320/2488320 [06:50<00:00, 6062.08it/s]

{0: 18, 1: 499175, 2: 1989127, 3: 0}
15 2
16 10
17 6

0.7 3
0.75 4
0.8 3
0.85 5
0.9 1
0.95 1
1.0 1


In [24]:
for a_list, output, r in rarity_vals[0.75]:
    print_results(a_list, output, r)
    print('')

--Pasture | Rarity: 0.75--
| Alligator - (Claw/Fang)
| Apkallu - (Fleece/Egg)
| Aurochs/Island Nanny - (Milk/Horn)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Blue Back/Dodo - (Egg/Feather)
| Chocobo - (Fur/Feather)
| Coblyn - (Fang/Carapace)
| Coblyn - (Fang/Carapace)
| Dodo of Paradise/Gold Back/Alkonost - (Feather/Egg)
| Glyptodon - (Claw/Carapace)
| Island Billy - (Horn/Fleece)
| Island Doe - (Fur/Milk)
| Island Doe - (Fur/Milk)
| Lost Lamb - (Fleece/Milk)
| Ornery Karakul - (Milk/Fleece)
| Paissa - (Claw/Fleece)
| Tiger/Quartz Spriggan/Weird Spriggan - (Fang/Fur)
| Yellow Coblyn/Morbol - (Carapace/Fang)
| Yellow Coblyn/Morbol - (Carapace/Fang)
--Avg Leavings per Day
| Carapace-5.5 | Guaranteed: 4 | Bonus: 3
| Fang-4.5 | Guaranteed: 3 | Bonus: 3
| Claw-4.0 | Guaranteed: 3 | Bonus: 2
| Milk-3.5 | Guaranteed: 2 | Bonus: 3
| Fur-3.5 | Guaranteed: 3 | Bonus: 1
| Fleece-3.5 | Guaranteed: 2 | Bonus: 3
| Feather-2.0 | Guaranteed: 1 | Bonus: 2


In [26]:
for a_list, output, r in best_setups[min_v]:
    print_results(a_list, output, r)
    print('')

--Pasture | Rarity: 0.8--
| Alligator - (Claw/Fang)
| Apkallu - (Fleece/Egg)
| Apkallu of Paradise - (Egg/Fleece)
| Aurochs/Island Nanny - (Milk/Horn)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Chocobo - (Fur/Feather)
| Chocobo - (Fur/Feather)
| Coblyn - (Fang/Carapace)
| Coblyn - (Fang/Carapace)
| Coblyn - (Fang/Carapace)
| Dodo of Paradise/Gold Back/Alkonost - (Feather/Egg)
| Grand Buffalo - (Horn/Milk)
| Ground Squirrel/Opo-Opo - (Claw/Fur)
| Island Doe - (Fur/Milk)
| Lost Lamb - (Fleece/Milk)
| Ornery Karakul - (Milk/Fleece)
| Paissa - (Claw/Fleece)
| Yellow Coblyn/Morbol - (Carapace/Fang)
| Yellow Coblyn/Morbol - (Carapace/Fang)
--Avg Leavings per Day
| Carapace-5.5 | Guaranteed: 4 | Bonus: 3
| Fang-4.5 | Guaranteed: 3 | Bonus: 3
| Claw-4.0 | Guaranteed: 3 | Bonus: 2
| Milk-3.5 | Guaranteed: 2 | Bonus: 3
| Fur-3.5 | Guaranteed: 3 | Bonus: 1
| Fleece-3.5 | Guaranteed: 2 | Bonus: 3
| Feather-2.0 | Guaranteed: 1 | Bonus: 2
| Egg-2.0 | Gu